In [1]:
import pandas as pd
import numpy as np


In [2]:
X = np.array(
    [['youth', 'high', 'no', 'fair'],
    ['youth', 'high', 'no', 'excellent'],
    ['middle_aged', 'high', 'no', 'fair'],
    ['senior', 'medium', 'no', 'fair'],
    ['senior', 'low', 'yes', 'fair'],
    ['senior', 'low', 'yes', 'excellent'],
    ['middle_aged', 'low', 'yes', 'excellent'],
    ['youth', 'medium', 'no', 'fair'],
    ['youth', 'low', 'yes', 'fair'],
    ['senior', 'medium', 'yes', 'fair'],
    ['youth', 'medium', 'yes', 'excellent'],
    ['middle_aged', 'medium', 'no', 'excellent'],
    ['middle_aged', 'high', 'yes', 'fair'],
    ['senior', 'medium', 'no', 'excellent']]
)

Y = np.array(['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no'])

X_learn = X[:-4]
Y_learn = Y[:-4]


X_test = X[-4:]
Y_test = Y[-4:]

column_labels = ['age', 'income', 'student', 'credit_rating']
df = pd.DataFrame(X_learn, columns=column_labels)

X_test_df = pd.DataFrame(X_test, columns=column_labels)

In [3]:
def accuracy(rule, positive, negative):
  P = np.sum(apply_rule(rule, positive))
  N = np.sum(apply_rule(rule, negative))

  print("SUMMMMMMMMMSSSS", P , N)

  if P + N == 0:
    return 0

  return (P) / (P + N)

def apply_rule(rule, data):
  matches = np.array([True] * len(data))

  for feature, value in rule:
    matches &= (data[feature] == value)

  return matches


In [4]:
def get_rule(positive, negative):
  conditions = []
  best_accuracy = 0

  # adding new rules
  for feature in positive.columns:
    for value in np.unique(positive[feature]):
      new_conditions = conditions + [(feature, value)];

      new_accuracy = accuracy(new_conditions, positive, negative)

      if new_accuracy > best_accuracy:
        best_accuracy = new_accuracy
        conditions = new_conditions
  print(conditions)

  # pruning rules
  for condition in conditions:
    new_conditions = [c for c in conditions if c != condition]
    new_accuracy = accuracy(new_conditions, positive, negative)

    if new_accuracy > best_accuracy and len(conditions) > 0:
      best_accuracy = new_accuracy
      conditions = new_conditions

  return conditions


In [5]:
def rip(X, Y):
  X = X.copy()
  Y = Y.copy()

  rules = []
  for c in np.unique(Y):
    class_rules = []
    i = 0

    positive = X[Y == c]
    negative = X[Y != c]
    while len(positive) != 0:
      print(positive, negative)
      rule = get_rule(positive, negative)
      print("RULE", rule)

      class_rules.append(rule)

      positive = positive[~apply_rule(rule, positive)]
      print("pos at the end", positive)

    rules.append((c, class_rules))
    print("overall rules")

  return rules

def predict(X, rules):
  predictions = []
  for _, row in X.iterrows():
      predicted_class = None

      for cls, rs in rules:
          for rule in rs:
              if all(row[feature] == value for feature, value in rule):
                  predicted_class = cls
                  break
          if predicted_class is not None:
              break

      if predicted_class is None:
          predicted_class = 'Unknown'

      predictions.append(predicted_class)
  return predictions

In [6]:
rules = rip(df, Y_learn)

print(rules)

print(predict(X_test_df, rules))


      age  income student credit_rating
0   youth    high      no          fair
1   youth    high      no     excellent
5  senior     low     yes     excellent
7   youth  medium      no          fair            age  income student credit_rating
2  middle_aged    high      no          fair
3       senior  medium      no          fair
4       senior     low     yes          fair
6  middle_aged     low     yes     excellent
8        youth     low     yes          fair
9       senior  medium     yes          fair
SUMMMMMMMMMSSSS 1 3
SUMMMMMMMMMSSSS 0 0
SUMMMMMMMMMSSSS 0 0
SUMMMMMMMMMSSSS 1 1
SUMMMMMMMMMSSSS 0 0
SUMMMMMMMMMSSSS 0 0
SUMMMMMMMMMSSSS 1 1
SUMMMMMMMMMSSSS 1 0
SUMMMMMMMMMSSSS 0 0
[('age', 'senior'), ('income', 'low'), ('credit_rating', 'excellent')]
SUMMMMMMMMMSSSS 1 1
SUMMMMMMMMMSSSS 1 0
SUMMMMMMMMMSSSS 1 1
RULE [('age', 'senior'), ('income', 'low'), ('credit_rating', 'excellent')]
pos at the end      age  income student credit_rating
0  youth    high      no          fair
1  yo